<a href="https://colab.research.google.com/github/DerNiccoo/TheEarlyBird/blob/main/Aufgabe6/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install box2d-py

     |████████████████████████████████| 450kB 4.3MB/s 


In [2]:
import gym
import random
import torch
import numpy as np
from collections import namedtuple, deque
import matplotlib.pyplot as plt
%matplotlib inline

# pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Initialisieren der Umgebung



In [3]:
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of Actions: ', env.action_space.n)

State shape:  (8,)
Number of Actions:  4


1. Define a neural network with two fully connected-layers. The hidden layer uses a
Relu activation. The output layer uses a softmax. Try different hidden layer sizes
(between 100 and 500). The network takes as input a vector of the current states
and gives out a probability for each action.

In [58]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, state_size, hidden_layer, action_size):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(state_size, hidden_layer)
        self.linear2 = torch.nn.Linear(hidden_layer, action_size)

    def forward(self, state):
        #Transformiere state (numpay array) zu Torch Tensor, da TwoLayerNet Tensor als Input erwartet
        state = torch.from_numpy(state)
        h_relu = F.relu(self.linear1(state))
        action_pred = F.softmax(self.linear2(h_relu))
        return action_pred

2. Generate 100 episodes by sampling actions using the network output. Limit the
number of steps per episode to 500. Sum up the reward of all steps of one episode.

In [77]:
def play_episodes(max_steps, model): 

    t = 0
    done = False
    action_space = [0, 1, 2, 3]

    #State-Action-Pairs pro Zeitschritt einer Episode
    state_action_per_timestep = []
    #Reward pro Zeitschritt einer Episode
    reward_per_timestep = []
     
    state = env.reset() #State ist ein 1-D Numpy Array mit 8 Einträgen 

    #Erstellung einer Episode
    while t < max_steps: 

      #print("STATE:::::::: ", state)
      #print("STATE TYPE + SHAPE ::::::::::::", state.shape, type(state) )
        
      action_output = model(state)
      #print("ACTIONS OUTPUT::::::::::::::", action_output)
      sampled_action = random.choices(action_space, action_output, k=1)
      #print("CHOOSEN ACTION:::::::::.", sampled_action[0])

      state_action_per_timestep.append((state, sampled_action[0]))

      next_state, reward, done, _ = env.step(sampled_action[0])
      reward_per_timestep.append(reward)

      state = next_state
      t += 1
     
      if done: 
        break
    
    return state_action_per_timestep, reward_per_timestep, t

In [82]:
episode_n = 100
max_steps = 500

#Rewards aller n Episoden
rewards_per_episode = []
#Durchschnittlicher Reward einer Episode
mean_rewards_per_episode = []

state_size, action_size, hidden_layer = 8, 64, 4
model = TwoLayerNet(state_size, action_size, hidden_layer)

for i in range(0, episode_n):
  state_actions, reward, episode_length = play_episodes(max_steps, model)
 
  #reward_per_episode = Liste mit allen summierten Rewards von 100 Episoden 
  rewards_per_episode.append((sum(reward), state_actions))
  mean_rewards_per_episode.append((sum(reward)/episode_length, state_actions))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


3. Print out the mean reward per episode of the 100 episodes.

In [80]:
#mean_rewards_per_episode

4. Identify the 20 best of those episodes in terms of reward and use the
<state, action> pairs of these episodes as training examples for the network.

In [124]:
#Sortiere Rewards absteigend
reward_sorted = sorted(rewards_per_episode)
mean_rewards_sorted = sorted(mean_rewards_per_episode)

#Schneide die besten 20 ab und extrahiere die State,actions pairs dieser Episoden 
top20 = reward_sorted[:20]
training_pairs = [x[1] for x in top20]

#print(training_pairs)

In [122]:


for pair in training_pairs:
  print(pair[0][0])
  #print(pair[1])


[ 0.00491934  1.4167392   0.4982709   0.25862202 -0.0056936  -0.11286581
  0.          0.        ]
[-0.00466557  1.4025742  -0.47258395 -0.37093422  0.005413    0.1070473
  0.          0.        ]
[ 0.00260124  1.4138703   0.26345855  0.13112561 -0.00300736 -0.05967727
  0.          0.        ]
[-0.00428696  1.3993648  -0.434241   -0.5135697   0.00497432  0.09836202
  0.          0.        ]
[ 8.0671313e-04  1.4174215e+00  8.1699163e-02  2.8894600e-01
 -9.2802680e-04 -1.8506095e-02  0.0000000e+00  0.0000000e+00]
[ 0.00639868  1.4196268   0.64809746  0.38694793 -0.00740764 -0.14680363
  0.          0.        ]
[-0.00732298  1.4101491  -0.7417462  -0.03429733  0.00849221  0.16801645
  0.          0.        ]
[ 0.00447531  1.410691    0.45327792 -0.01018432 -0.00517889 -0.10267415
  0.          0.        ]
[ 1.3033866e-03  1.4203258e+00  1.3199742e-01  4.1802695e-01
 -1.5034557e-03 -2.9899385e-02  0.0000000e+00  0.0000000e+00]
[ 0.0043931   1.4187808   0.44496006  0.3493579  -0.00508375 -

In [175]:
def train(training_set, model):
  
  model.train()
  for pair in training_set:

    state = pair[0][0]
    target_action = pair[0][1]

    target = torch.zeros(1, dtype=torch.long)
    target[0] = target_action

    output_action = model(state).unsqueeze(0)
    #print(f"TARGET ACTIOM {target.shape}")
    #print(f"TARGET ACTIOM {output_action.shape}")

    loss = F.nll_loss(output_action, target)
    loss.backward()

In [173]:
train(training_pairs, model)

TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2387, grad_fn=<NllLossBackward>)
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2692, grad_fn=<NllLossBackward>)
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2697, grad_fn=<NllLossBackward>)
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2258, grad_fn=<NllLossBackward>)
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2285, grad_fn=<NllLossBackward>)
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2546, grad_fn=<NllLossBackward>)
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2579, grad_fn=<NllLossBackward>)
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2454, grad_fn=<NllLossBackward>)
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2773, g

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


In [174]:
mean_reward = 0 # as Learning Indicator
episode_n = 100
max_steps = 500

#Definiere das Neuronale Netz mit einer Hidden_Layer
state_size, hidden_layer, action_size = 8, 64, 4
model = TwoLayerNet(state_size, hidden_layer, action_size)

while mean_reward < 100: 

  #2: Generate 100 episodes by sampling actions using the network output
  
  #Rewards aller n Episoden
  rewards_per_episode = []
  #Durchschnittlicher Reward einer Episode
  mean_rewards_per_episode = []

  for i in range(0, episode_n):
    state_actions, reward, episode_length = play_episodes(max_steps, model)
  
    mean_reward = sum(reward) / episode_length
    sum_reward = sum(reward) 
    
    #reward_per_episode = Liste mit allen summierten Rewards von 100 Episoden
    rewards_per_episode.append((sum_reward, state_actions))
    mean_rewards_per_episode.append((mean_reward, state_actions))

    #3: Print out the mean reward per episode of the 100 episodes.
    print(f"Mean Reward in Episode {i}: {mean_reward}")
  
  #4: Identify the 20 best of those episodes in terms of reward and use the <state, action> pairs of these episodes as training examples for the network.
  #Sortiere Rewards absteigend
  reward_sorted = sorted(rewards_per_episode)
  
  #Schneide die besten 20 ab und extrahiere die State,actions pairs dieser Episoden 
  top20 = reward_sorted[:20]
  training_pairs = [x[1] for x in top20]

  train(training_pairs, model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.2389, grad_fn=<NllLossBackward>)
TARGET ACTIOM torch.Size([1])
TARGET ACTIOM torch.Size([1, 4])
LOSSSSS tensor(-0.3148, grad_fn=<NllLossBackward>)
Mean Reward in Episode 0: -2.9255457988371765
Mean Reward in Episode 1: -4.844163002086325
Mean Reward in Episode 2: -1.3902082939581077
Mean Reward in Episode 3: -0.8695420878226007
Mean Reward in Episode 4: -1.0608833045172517
Mean Reward in Episode 5: -1.440458687846641
Mean Reward in Episode 6: -3.672073022156297
Mean Reward in Episode 7: -0.9702208767381253
Mean Reward in Episode 8: -1.1984866120120583
Mean Reward in Episode 9: -2.8212644372224873
Mean Reward in Episode 10: -1.6390654641815396
Mean Reward in Episode 11: -1.0622972945017666
Mean Reward in Episode 12: -0.07888719819355092
Mean Reward in Episode 13: -2.2478755775260857
Mean Reward in Episode 14: -3.6223556758836786
Mean Reward 

KeyboardInterrupt: ignored

In [97]:
test = []
test.append(([0,2,3,],1))

np.array(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


array([[list([0, 2, 3]), 1]], dtype=object)